# Rede Neural de Base Radial (RBF)

As redes RBF são redes de alimentação direta (feedforward) consistindo de três camadas:


1.   **Camada de entrada**: propaga os estímulos
2.   **Camada escondida**: Unidades de processamento localmente sintonizáveis, utilizando mapeamento não linear.
3.   **Camada de saída**: Unidades de processamento lineares.


****

**O treinamento dessa rede ocorre de forma híbrida**, combinando aprendizagem não supervisionada (ANS) com a supervisionada(AS). Isso ocorre, pois em geral não se sabe quais saídas se desejam para a camada escondida. Sendo assim, a distribuição de trabalhos ocorre:
*   **ANS**: Treina a camada escondida definindo seus parâmetros livres (centros, larguras dos campos receptivos e pesos).
*   **AS**: Determina os valores dos pesos entre as camadas escondidas e de saída, considerando constantes os parâmetros já definidos.


****

**O aprendizado consiste em** determinar os valores para:
*   centro das funções de base radial,
*   largura das funções,
*   pesos da camada de saída.


Além disso, para cada neurônio da camada escondida, ele computa uma função de base radial.


Os passos necessários são:
1.   Utilizar um algoritmo ANS para encontrar os centros (protótipo para um cluster) das RBF;
2.   Utilizar métodos heurísticos para determinar a largura (área de influência de um cluster) de cada função;
3.   Utilizar um AS para determinar os pesos da camada de saída da rede.

1ª Etapa: Inicialização dos grupos com K-Means

In [1]:
!git clone https://github.com/valmirf/redes_neurais_pos.git

Cloning into 'redes_neurais_pos'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 120 (delta 11), reused 1 (delta 0), pack-reused 90 (from 1)
Receiving objects: 100% (120/120), 16.66 MiB | 388.00 KiB/s, done.
Resolving deltas: 100% (35/35), done.


Definição da função de base radial

In [2]:
#função de base radial gaussiana
def rbfGaussiana(x, c, s):
    return np.exp(-1 / (2 * s**2) * (x-c)**2)

def rbfMultiquadratica(x, c, s):
    return 1 / np.sqrt((x - c) ** 2 + s ** 2)

#função de cálculo da largura do campo receptivo em que se repete o mesmo valor pra todos os neurônios
def computeEqualStds(centers,k):
  dist = [np.sqrt(np.sum((c1 - c2) ** 2)) for c1 in centers for c2 in centers]
  dMax = np.max(dist)
  stds = np.repeat(dMax / np.sqrt(2 * k), k)
  return stds

def computeIndividualStds(centers):
    stds = []
    for i, c1 in enumerate(centers):
        distances = [np.linalg.norm(c1 - c2) for j, c2 in enumerate(centers) if i != j] #calcula a distancia media entre o centro c1 e todos os outros centros
        stds.append(np.mean(distances))  #media das distancias como largura
    return np.array(stds)

2ª Etapa - Treinamento de uma Rede Neural

In [3]:
from sklearn.cluster import KMeans
import numpy as np

class RBFNet(object):
    """Implementation of a Radial Basis Function Network"""

    def __init__(self, k=3, attnumber=4, lr=0.01, epochs=100, rbf=rbfMultiquadratica, computeStds=computeIndividualStds):
        self.k = k  # grupos ou numero de neuronios na camada escondida
        self.lr = lr # taxa de aprendizagem
        self.epochs = epochs  # número de iterações
        self.rbf = rbf # função de base radial
        self.computeStds = computeStds  #função de cálculo da largura do campo receptivo

        self.w = np.random.randn(self.k,attnumber)
        self.b = np.random.randn(1)

    def fit(self, X, y):
        self.stds = []
        #K-Means pra pegar os centros inicias
        #1º parâmetro da rede RBF
        kmeans = KMeans(
            n_clusters=self.k, init='random',
            n_init=10, max_iter=300).fit(X)
        self.centers = kmeans.cluster_centers_
        #print('centers: ', self.centers)

        #Cálculo la dargura do campo receptivo
        #2º parâmetro da rede RBF
        #self.stds = self.computeStds(self.centers)
        if self.computeStds == computeEqualStds:
            self.stds = self.computeStds(self.centers, self.k)
        else:
            self.stds = self.computeStds(self.centers)

        # training
        for epoch in range(self.epochs):
            for i in range(X.shape[0]):
                # forward pass
                #calcula a saída de cada neurônio da função de base radial
                phi = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
                #calcula somatório do produto da saída da função de base radial e os pesos
                F = phi.T.dot(self.w)
                F = np.sum(F) + self.b
                #saída da rede
                out = 0 if F < 0 else 1

                #função de perda
                loss = (y[i] - out).flatten() ** 2
                #print('Loss: {0:.2f}'.format(loss[0]))

                #cálculo do erro
                error = (y[i] - out).flatten()
                #atualização dos pesos
                #3º Parâmetro da rede
                self.w = self.w + self.lr * error * phi
                self.b = self.b + self.lr * error

    #calcula saída da rede RBF com a rede treinada
    def predict(self, X):
        y_pred = []
        error = 0
        for i in range(X.shape[0]):
            a = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
            F = a.T.dot(self.w)
            F = np.sum(F) + self.b
            out = 0 if F < 0 else 1
            y_pred.append(out)

        return np.array(y_pred)




```
# Isto está formatado como código
```

# Executando a rede neural RBF

In [4]:
# Neste código vou utilizar o pandas, framework amplamente utilizado pra lidar com dados
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#carrega a base de dados e retorna conjuntos de treinamento e teste
def load_data():
    url = 'redes_neurais_pos/RBF/diabetes.csv'
    df = pd.read_csv(url)
    #remove a ultima coluna (dados)
    data = df[df.columns[:-1]]
    #normaliza os dados
    normalized_data = (data - data.min()) / (data.max() - data.min())
    #retorna a última coluna (rótulos)
    labels = df[df.columns[-1]]
    #separa em conjunto de treinamento e teste com seus respectivos rótulos
    X_train, X_test, y_train, y_test = train_test_split(normalized_data, labels, test_size=0.2, random_state=0)

    return X_train, X_test, y_train, y_test

#chama função que carrega base de dados
#chama função que carrega base de dados
training_inputs, test_inputs, training_labels, test_labels = load_data()

# Verifica o número de atributos
attnumber = training_inputs.shape[1]  # Número de atributos

#transforma rótulos do conjunto de treinamento em numeros pra calculo do erro
le = preprocessing.LabelEncoder()
le.fit(training_labels.values)
training_labels_transformed = le.transform(training_labels.values)

# CODIGO DA QUESTAO 1
neurons_options = [3, 5, 7, 9] #neurônios escondidos
errors = {}

for k in neurons_options:
    rbfnet = RBFNet(lr=1e-2, attnumber=attnumber, k=k, computeStds=computeEqualStds)
    rbfnet.fit(training_inputs.values, training_labels_transformed)

    # aqui transformamos os rotulos do conjunto de teste em numeros para calculo do erro
    test_labels_transformed = le.transform(test_labels.values)
    y_pred = rbfnet.predict(test_inputs.values)

    errorabs = abs(test_labels_transformed - y_pred)
    error_rate = np.sum(errorabs) / len(test_labels_transformed)
    errors[k] = error_rate
    print(f'Taxa de erro com {k} neuronios escondidos: {error_rate:.4f}')


# CODIGO DA QUESTAO 2
#Usando a melhor configuração de 9 neurônios escondidos
# k = 9
# rbfnet = RBFNet(lr=1e-2, attnumber=attnumber, k=k, computeStds=computeIndividualStds)
# rbfnet.fit(training_inputs.values, training_labels_transformed)

# # Transforma rótulos do conjunto de teste em números para cálculo do erro
# test_labels_transformed = le.transform(test_labels.values)
# y_pred = rbfnet.predict(test_inputs.values)

# errorabs = abs(test_labels_transformed - y_pred)
# error_rate = np.sum(errorabs) / len(test_labels_transformed)

# print(f'Taxa de erro com larguras individuais: {error_rate:.4f}')

#CODIGO DA QUESTAO 3
# Define as configurações para combinar
rbf_functions = [rbfGaussiana, rbfMultiquadratica]
std_methods = [computeEqualStds, computeIndividualStds]
configurations = [(rbf_func, std_method) for rbf_func in rbf_functions for std_method in std_methods]

# Lista de números de neurônios escondidos para teste
neurons_options = [3, 5, 7, 9]
results = {}

for k in neurons_options:
    print(f"\nNúmero de neurônios escondidos: {k}")
    for rbf_func, std_method in configurations:
        rbf_func_name = rbf_func.__name__
        std_method_name = std_method.__name__

        # Inicializa e treina a rede RBF
        rbfnet = RBFNet(lr=1e-2, attnumber=attnumber, k=k, rbf=rbf_func, computeStds=std_method)
        rbfnet.fit(training_inputs.values, training_labels_transformed)

        # Calcula a taxa de erro no conjunto de teste
        test_labels_transformed = le.transform(test_labels.values)
        y_pred = rbfnet.predict(test_inputs.values)

        errorabs = abs(test_labels_transformed - y_pred)
        error_rate = np.sum(errorabs) / len(test_labels_transformed)

        # Exibe os resultados
        print(f"RBF: {rbf_func_name}, Método Largura: {std_method_name}, Taxa de erro: {error_rate:.4f}")
        results[(k, rbf_func_name, std_method_name)] = error_rate

# Determina a melhor configuração
best_config = min(results, key=results.get)
print("\nMelhor configuração encontrada:")
print(f"Neurônios escondidos: {best_config[0]}")
print(f"Função de Base Radial: {best_config[1]}")
print(f"Método de Cálculo da Largura: {best_config[2]}")
print(f"Taxa de erro: {results[best_config]:.4f}")


Taxa de erro com 3 neuronios escondidos: 0.3117
Taxa de erro com 5 neuronios escondidos: 0.2143
Taxa de erro com 7 neuronios escondidos: 0.3442
Taxa de erro com 9 neuronios escondidos: 0.2857

Número de neurônios escondidos: 3
RBF: rbfGaussiana, Método Largura: computeEqualStds, Taxa de erro: 0.2792
RBF: rbfGaussiana, Método Largura: computeIndividualStds, Taxa de erro: 0.2338
RBF: rbfMultiquadratica, Método Largura: computeEqualStds, Taxa de erro: 0.3117
RBF: rbfMultiquadratica, Método Largura: computeIndividualStds, Taxa de erro: 0.2273

Número de neurônios escondidos: 5
RBF: rbfGaussiana, Método Largura: computeEqualStds, Taxa de erro: 0.2857
RBF: rbfGaussiana, Método Largura: computeIndividualStds, Taxa de erro: 0.2208
RBF: rbfMultiquadratica, Método Largura: computeEqualStds, Taxa de erro: 0.3312
RBF: rbfMultiquadratica, Método Largura: computeIndividualStds, Taxa de erro: 0.2208

Número de neurônios escondidos: 7
RBF: rbfGaussiana, Método Largura: computeEqualStds, Taxa de erro: 

# Descrição Mini Projeto

Utilizando o código acima, modifique a última seção (Executando com Base de Dados) para que ele seja executado com a base de dados do arquivo diabetes.csv. Depois, modifique a função de base radial implementada (Gaussiana) para a Multiquadrática Inversa e calcule a taxa de erro.


1- Calcular a quantidade de neurônios escondidos:

a) 3

b) 5

c) 7

d) 9

Qual foi a melhor configuração? Avaliaria um outro valor?

R -> A melhor configuracao foi com 9 neuronios escondidos, com uma taxa de erro de 18,83%.

2- Utilizando a melhor configuração da questão anterior, calcular a taxa de erro usando uma das outras maneiras de retorno da largura do campo receptivo da função de base radial, em que cada neurônio possui sua própria largura.

R -> Taxa de erro com larguras individuais: 34,42%

3- Calcular a taxa de erro combinando 2 funções de Base Radial e as duas maneiras de cálculo da largura do campo receptivo:

a) Gaussiana

b) Multiquadrática Inversa


Qual foi a melhor configuração?

R -> A melhor configuracao encontrada foi a rbfMultiquadratica como funcao de base radial, com 5 neuronios escondidos e usando o computeEqualStds como metodo de calculo de largura. A taxa de erro foi de 19,48%.
